In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import torch
# from fake_claims_generator import generate, grab_random_prompts

In [ ]:
!pip3 install git+https://github.com/huggingface/transformers@v3.1.0 # there are errors in the later versions

  Cloning https://github.com/huggingface/transformers (to revision v3.1.0) to /tmp/pip-req-build-zv6l46c3
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-zv6l46c3
  Running command git checkout -q 4f0a753715e2a4b521db2e91200d947114e0c90c
     |████████████████████████████████| 3.0MB 6.5MB/s 
     |████████████████████████████████| 1.1MB 38.6MB/s 
     |████████████████████████████████| 890kB 27.8MB/s 
  Created wheel for transformers: filename=transformers-3.1.0-cp36-none-any.whl size=884019 sha256=bcd6382bbfe8d80916077da46fb0a6a3454fd38f9a46c4e59b6961f07a8cd84f
  Stored in directory: /tmp/pip-ephem-wheel-cache-__or5aam/wheels/97/58/f3/fab46f0e4c13b26fe3bb745b20a17792a12b324163b2cfe393
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=5eb67d8cd88700d68e70d0a8e61a27edf93b76471f13e62e5b8184e6d4058d51
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
%cd drive/My\ Drive/School\ Work/Fourth\ Year/Capstone\ Everything

/content/drive/My Drive/School Work/Fourth Year/Capstone Everything


In [ ]:
def grab_random_prompts(data, numprompts, first_words=5, column='claims'):
    '''
    Inputs:
    numprompts int: Number of prompts we would want
    first_words int: How many first words would we want
    '''
    num_data = len(data)
    prompts = []
    while len(prompts) < numprompts:
        row = np.random.randint(0, num_data-1)
        try:
            words = data[column][row].split()
            firsts = " ".join(words[:first_words])
            prompts.append(firsts)
        except:
            #print("No claims")
            pass
    
#     for indx in rows:
#         words = data[column][indx].split()
#         firsts = " ".join(words[:first_words])
#         prompts.append(firsts)
    return prompts

OUTPUT_DIR = "./Models/output"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)
def generate(input_str, length=250, n=5):
    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
    model.eval()
    with torch.no_grad():
        for i in range(length):
            outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0)
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        return output_text

def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [ ]:
device

'cuda'

## Load Real Data and Label Them


In [ ]:
data = pd.read_csv("./Models/Data/H04L.csv")
real_data = pd.DataFrame({"Claims": data["claims"]})
real_data["Label"] = 0
real_data = real_data[real_data['Claims'].notnull()]

In [ ]:
real_data

,Claims,Label
0,1. An organic light emitting diode display co...,0
1,1. An electric element cooling module compris...,0
2,2. The organic light emitting diode display o...,0
3,2. The electric element cooling module of cla...,0
4,3. The organic light emitting diode display o...,0
...,...,...
8941,"2. The method of claim 1, further comprising:...",0
8942,"3. The method of claim 1, wherein the first g...",0
8943,"4. The method of claim 1, wherein a width of ...",0
8944,"5. The method of claim 1, wherein a length of...",0


## Generate Fake Data and Label Them

In [ ]:
# How many fake claims do you want to generate
# num_fake_claims = len(real_data)
# prompts = grab_random_prompts(data, num_fake_claims, first_words=5)
# fake_claims = []
# for i, prompt in enumerate(prompts):
#     print(f"Generating Prompt {i+1} ... Prompt: {prompt}")
#     generated_text = generate(prompt, n=8)
#     fake_claims.append(generated_text)
  

In [ ]:
# write in chunks, and write to csv
'''
This code block is responsible for generating fake patent Claims
I first deine the file paths where we will save the fake patent data
Then, I generate the fake patent prompts by sampling from the real abstracts
I save the prompts into a csv file.
Since I am running the generation on Colab and I'm generating thousands of fake patent Claims,
there is a high frequency of interuption, I therefore need to save the prompts (to be accessed later),
and save the patent claims for every 100 that are generated.
The last_interupted variable stores where the generation was last interupted.
'''
import os.path
fake_data_file = "Models/Data/real_fake_patent_claims/fake_patent_claims.csv"
prompt_file = "Models/Data/real_fake_patent_claims/prompts.csv"
num_fake_claims = len(real_data)
if os.path.exists(prompt_file):
  prompts = pd.read_csv(prompt_file)
  prompts = prompts['Prompts'].tolist()
else:
  prompts = grab_random_prompts(data, num_fake_claims, first_words=5)
  # save prompts
  prompt_df = pd.DataFrame({"Prompts": prompts})
  prompt_df.to_csv(prompt_file)
fake_claims = []
last_interupted = 8400
for i, prompt in enumerate(prompts[last_interupted:]): # modify last_interupted if we got disconnected
    if i % 100 == 0 and i != 0: # for every 100 claims
      df = pd.DataFrame({"Claims": fake_claims, "Label": 1})
      if os.path.exists(fake_data_file): # this means this csv is already created
        df.to_csv(fake_data_file, mode='a', header=False)
      else:
        df.to_csv(fake_data_file)
      fake_claims = []
    print(f"Generating Prompt {i+1} ... Prompt: {prompt}")
    generated_text = generate(prompt, n=8)
    fake_claims.append(generated_text)
# last batch
df = pd.DataFrame({"Claims": fake_claims, "Label": 1})
df.to_csv(fake_data_file, mode='a', header=False)

Generating Prompt 1 ... Prompt: 5. The display panel according
Generating Prompt 2 ... Prompt: 35. The electrical circuit of
Generating Prompt 3 ... Prompt: 2. The method of claim
Generating Prompt 4 ... Prompt: 20. The emitter of claim
Generating Prompt 5 ... Prompt: 3. The device according to
Generating Prompt 6 ... Prompt: 31. The display apparatus according
Generating Prompt 7 ... Prompt: 11. The organic light-emitting device
Generating Prompt 8 ... Prompt: 9. An imprint lithography system,
Generating Prompt 9 ... Prompt: 10. An unmanned aerial vehicle,
Generating Prompt 10 ... Prompt: 3. The method for manufacturing
Generating Prompt 11 ... Prompt: 23. A semiconductor die comprising:
Generating Prompt 12 ... Prompt: 9. An anti-tamper system according
Generating Prompt 13 ... Prompt: 3. The wafer level package
Generating Prompt 14 ... Prompt: 17. The computing device of
Generating Prompt 15 ... Prompt: 3. The organic electric element
Generating Prompt 16 ... Prompt: 15. The method 

In [ ]:
fake_data = pd.read_csv("./Models/Data/real_fake_patent_claims/fake_patent_claims.csv")
fake_data = fake_data.loc[:, ~fake_data.columns.str.contains('^Unnamed')]
fake_data.head()


,Claims,Label
0,11. A substrate container comprising: (1) a ...,1
1,20. The flexible display according to embodime...,1
2,20. The packaging method of the device i...,1
3,3. The method of claim relates to a resin comp...,1
4,11. The package structure as a whole and the ...,1


In [ ]:
len(fake_data)

8420

In [ ]:
frames = [real_data, fake_data]
full_data = pd.concat(frames)


full_data = full_data.sample(frac=1).reset_index(drop=True)


In [ ]:
full_data


,Claims,Label
0,9. A supporting method performed by a support...,0
1,9. A pixel assembly according to one embodimen...,1
2,"11. The UV light emitting device of claim 1, ...",0
3,14. The method of claim 13 wherein planarizin...,0
4,"7. The display apparatus of claim 1, wherein ...",0
...,...,...
16835,17. The method as claimed may comprise: provid...,1
16836,"36. The ASIC of claim 24, wherein: at least a...",0
16837,5. The electronic component mounting bracket a...,1
16838,9. The vibration-type actuator according to c...,0


In [ ]:
full_data.to_csv("./Models/Data/real_fake_patent_claims/real_fake_claims.csv")